In [1]:
from huggingface_hub import notebook_login
notebook_login()


In [2]:
from datasets import load_dataset
eli5 = load_dataset("dany0407/eli5_category", split = "train[:5000]")


In [3]:
eli5 = eli5.train_test_split(test_size = 0.2)

In [4]:
eli5["train"][0]


{'q_id': '767kyu',
 'title': "How does a radio station send metadata like song name to my car's radio?",
 'selftext': "I was driving in my car, and for the first time noticed that the name of the current song on the radio was scrolling across the little LCD screen, and I wondered how that metadata works from the radio station's perspective.",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dobwoc6', 'doc0znu'],
  'text': ['Modulation became more efficient, which opened up space in the guard bands to transmit digital information on a sub carrier. Edit : okay imagine that while recording the song the radio station was tapping out a special code in the background of the song that only your car radio could hear. That code is the meta data.',
   "You're seeing the effects of the Radio Data System (RDS). Basically the radio stations encode simple data as a series of a high-frequency (inaudible to humans) tones which a decoder in your radio can detect and 

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

In [6]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '767kyu',
 'title': "How does a radio station send metadata like song name to my car's radio?",
 'selftext': "I was driving in my car, and for the first time noticed that the name of the current song on the radio was scrolling across the little LCD screen, and I wondered how that metadata works from the radio station's perspective.",
 'category': 'Technology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dobwoc6', 'doc0znu'],
 'answers.text': ['Modulation became more efficient, which opened up space in the guard bands to transmit digital information on a sub carrier. Edit : okay imagine that while recording the song the radio station was tapping out a special code in the background of the song that only your car radio could hear. That code is the meta data.',
  "You're seeing the effects of the Radio Data System (RDS). Basically the radio stations encode simple data as a series of a high-frequency (inaudible to humans) tones which a decoder in your radio can detect an

In [7]:
def preprocess_function(example):
    return tokenizer([" ".join(x) for x in example["answers.text"]])

In [8]:
tokenized_eli5 = eli5.map(
    preprocess_function, 
    batched = True, 
    num_proc = 4, 
    remove_columns = eli5["train"].column_names, 
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1646 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1079 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1629 > 1024). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1123 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1197 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1393 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3437 > 1024). Running this sequence through the model will result in indexing errors


In [9]:
block_size = 128
def group_texts(examples):
    concatenated_examples = {k : sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    
    result = {
        k :[t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc = 4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForLanguageModeling
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

In [12]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

In [13]:
training_args = TrainingArguments(
    output_dir = "first_eli5_clm-model", 
    eval_strategy = "epoch", 
    learning_rate = 2e-5, 
    weight_decay = 0.01, 
    push_to_hub = True, 
)

trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset = lm_dataset['train'],
    eval_dataset = lm_dataset['test'], 
    data_collator = data_collator, 
    tokenizer = tokenizer,
)

trainer.train()

/var/folders/bb/_7c_tp6x5yz_39tq724bm5pc0000gn/T/ipykernel_38488/4065628419.py:9: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.918000,3.842024
2,3.825900,3.831560
3,3.784500,3.829508


/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/pranamshetty/Developer/workshop/transformers-from-scratch/venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3921, training_loss=3.8455089830798905, metrics={'train_runtime': 972.6906, 'train_samples_per_second': 32.249, 'train_steps_per_second': 4.031, 'total_flos': 1024544559071232.0, 'train_loss': 3.8455089830798905, 'epoch': 3.0})

In [14]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity : {math.exp(eval_results['eval_loss']):.2f}")

Perplexity : 46.04


In [15]:
trainer.push_to_hub()

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...st_eli5_clm-model/training_args.bin: 100%|##########| 5.78kB / 5.78kB            

  ...st_eli5_clm-model/model.safetensors:   0%|          |  550kB /  328MB            

CommitInfo(commit_url='https://huggingface.co/prxshetty/first_eli5_clm-model/commit/97fc2fa725d5dc2fd8eff465129c1b578db356bc', commit_message='End of training', commit_description='', oid='97fc2fa725d5dc2fd8eff465129c1b578db356bc', pr_url=None, repo_url=RepoUrl('https://huggingface.co/prxshetty/first_eli5_clm-model', endpoint='https://huggingface.co', repo_type='model', repo_id='prxshetty/first_eli5_clm-model'), pr_revision=None, pr_num=None)

In [21]:
from transformers import pipeline
prompt = "Somatic hypermutation allows the immune system to"
output = pipeline("text-generation", model = "prxshetty/first_eli5_clm-model")
output(prompt)

Device set to use mps:0


[{'generated_text': 'Somatic hypermutation allows the immune system to "immediately" respond by "getting" their body to "expand" it. This can be achieved by using this technique called "intermediate immunization," which can be accomplished by the immune system\'s body using specialized techniques to control the immune systems\' responses. If you look at a tumor, you see a huge amount of cells around it and it\'s going to get bigger and larger. It\'s what causes cancer cells to grow, and that\'s why they\'re important. There are more cells in the tumor than normal, and we don\'t know how much they grow. Immunization works by giving the immune system "specific" immune responses to the cells, but not how much the immune system gets to. When you cut off the lymphatic system, it\'s not really that much to the brain, but to the brain as well. It\'s basically the same thing. "You can\'t survive without a normal cell to process all the different things it needs to survive," says David Blain. "

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("prxshetty/first_eli5_clm-model")
inputs = tokenizer(prompt, return_tensors="pt").input_ids

from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("prxshetty/first_eli5_clm-model")
outputs = model.generate(inputs, max_new_tokens = 100, do_sample = True, top_k = 50, top_p = 0.95)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [23]:
print(output)
tokenizer.batch_decode(outputs, skip_special_tokens = True)

["Somatic hypermutation allows the immune system to adapt to a more drastic response. When the immune system is overwhelmed, the immune system does NOT adapt to new stimuli (such as allergies, etc...) The reason the immune system doesn't adapt can be because the immune system is basically the same thing as our immune system. I am not a fan of that, but I can see how the body responds to any other form of extreme stress because the immune system is already working to adapt to extreme stress. There is actually a mechanism that works so well with"]